In [1]:
import pickle
import json

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

ANALISI CULTURA E SPETTACOLO

Utilizzeremo i seguenti dati: 

1. **Musei**
2. **Cinema**
3. **Teatri**

Saranno i nostri "poi", cioè "Point of Interest"
che verranno aggiunti al Grafo della rete 
Ciclabile/Stradale per la ricerca di percorsi

In [2]:
gdf_list = [
    {"gdf":  gpd.read_file(PATH_MUSEI_CLEAN),
     "tipo": "Musei",
     "gdf":  gpd.read_file(PATH_CINEMA_CLEAN),
     "tipo": "Cinema", 
     "gdf":  gpd.read_file(PATH_TEATRI_CLEAN),
     "tipo": "Teatri"}
     ]
Graph_functions.auto_analysis_poi(gdf_list, PATH_GEOJSON="../Data/Clean/Analisi/Cultura_e_spettacolo.geojson")

In [4]:
gdf_analisi_cultura_spettacolo = gpd.read_file("../Data/Clean/Analisi/Cultura_e_spettacolo.geojson", driver= "GeoJSON")

c:\Users\apera\OneDrive\Documenti\progetti\Progetto-Bike\.venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver GeoJSON does not support open option DRIVER
  return ogr_read(


In [9]:
# Eliminiamo connessioni "artificiali" usate per connettere i "poi" al grafo delle strade
gdf_analisi_cultura_spettacolo_mod = gdf_analisi_cultura_spettacolo[gdf_analisi_cultura_spettacolo["artificial"] != True]

# Leggi i due POI
gdf_musei = gpd.read_file(PATH_MUSEI_CLEAN)
gdf_teatri = gpd.read_file(PATH_TEATRI_CLEAN)
gdf_cinema = gpd.read_file(PATH_CINEMA_CLEAN)

# Unisci in un'unica geometria
geometry_poi = gpd.GeoSeries(
    gpd.pd.concat([gdf_cinema.geometry, gdf_teatri.geometry, gdf_musei.geometry], ignore_index=True)
).union_all()

# Filtra il GDF di partenza
gdf_senza_poi = []
for idx, row in gdf_analisi_cultura_spettacolo.iterrows():
    if not geometry_poi.contains(row.geometry):
        gdf_senza_poi.append(row)

# Crea nuovo GeoDataFrame
gdf_analisi_cultura_spettacolo_mod = gpd.GeoDataFrame(gdf_senza_poi, crs=CRS_GRAD)

# Salviamo per test di visualizzazione su kepler
gdf_analisi_cultura_spettacolo_mod.to_file("../Data/Clean/Analisi/Cultura_e_spettacolo_senza_poi.geojson", driver="GeoJSON")

# Raggruppiamo per "highway" e calcoliamo (in km -> /1000) la lunghezza complessiva delle strade
gdf_analisi_cultura_spettacolo_mod_most_highways = gdf_analisi_cultura_spettacolo_mod.to_crs(CRS_METR).groupby("highway").agg({
    "length": lambda x: sum(x/1000)
    }).sort_values(by="length", ascending=False)

In [10]:
# Stampiamo il risultato
print(gdf_analisi_cultura_spettacolo_mod_most_highways)
print(f"Tot: {gdf_analisi_cultura_spettacolo_mod_most_highways["length"].sum()}")

                             length
highway                            
cycleway                  32.205970
footway                   20.011504
residential                8.810256
tertiary                   4.989268
secondary                  4.598090
primary                    1.666067
pedestrian                 0.914740
footway steps              0.381345
service                    0.265643
unclassified               0.204832
secondary_link             0.172719
residential cycleway       0.116689
residential pedestrian     0.097656
primary_link               0.082050
footway service            0.078886
unclassified residential   0.066621
cycleway footway           0.058601
path                       0.001734
Tot: 74.7226731089275
